## Preprocessing

In [2]:
import gensim
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import *
# NLP with NLTK
import nltk
nltk.download("stopwords")
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import os
import raw_utils

import pandas as pd
import preprocessing as util

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/khanhnguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/khanhnguyen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'datasets/csv/')

phishing_csv = 'raw_phishing.csv'
legitimate_csv = 'raw_legitimate.csv'

In [4]:
phishing_text_raw = pd.read_csv(os.path.join(csv_path, phishing_csv), index_col=0, dtype={'body': 'object'})
legitimate_text_raw = pd.read_csv(os.path.join(csv_path, legitimate_csv), index_col=0, dtype={'body': 'object'})

In [5]:
phishing_text_raw.head()

,body
0,NaN
1,"Hello nrt5,\n\nGeekSquad® Support-Team +1-813-..."
2,\n<html>\n<head>\n\t<title></title>\n</head>\n...
3,\n \n<http://richellebrooks.com/Servsn.htm>
4,"jose@monkey.org, \n\nYour mailbox quota is ful..."


In [6]:
legitimate_text_raw['body']

0       Matthias Saou (matthias@rpmforge.net) wrote*:\...
1       |:::::::::::::::::::::::::::::::::::::::::::::...
2       Hello Bill,\n\nMonday, September 30, 2002, 5:4...
3       >>>Chris Garrigues said:\n > > From:  Brent We...
4       from slate's "today's papers":  \nThe New York...
                              ...                        
6573     <http://www.sportsline.com/links/6/1/163/sffe...
6574    \nAs much as I'd like to, I can't get to every...
6575    Darrell,\n\nBen at Conoco would like to know t...
6576    Just to keep you in the loop.   I am addressin...
6577    I was looking for tommorow.   I'll pass on ton...
Name: body, Length: 6578, dtype: object

## Cleanup

In [7]:
phishing_text_raw = phishing_text_raw.dropna()
phishing_text = phishing_text_raw[phishing_text_raw['body'].str.contains("This text is part of the internal format of your mail folder, and is not\na real message.") == False]
phishing_text = phishing_text[phishing_text.duplicated(keep='first') == False]
phishing_text.shape

(4927, 1)

In [8]:
legitimate_text_raw = legitimate_text_raw.dropna()
legitimate_text = legitimate_text_raw[legitimate_text_raw['body'].str.contains("This text is part of the internal format of your mail folder, and is not\na real message.") == False]
legitimate_text = legitimate_text[legitimate_text.duplicated(keep='first') == False]
legitimate_text.shape

(6373, 1)

In [9]:
phishing_text['body'] = phishing_text['body'].apply(util.replace_email)
phishing_text['body'] = phishing_text['body'].apply(util.replace_url)

In [10]:
legitimate_text['body'] = legitimate_text['body'].apply(util.replace_email)
legitimate_text['body'] = legitimate_text['body'].apply(util.replace_url)

In [11]:
# Custom stop words and preprocessing filters

stopWords = nltk.corpus.stopwords
stopWords = stopWords.words("english")
stopWords.extend(["nbsp", "font", "sans", "serif", "bold", "arial", "verdana", "helvetica", "http", "https", "www", "html", "enron", "margin", "spamassassin"])

def remove_custom_stopwords(p):
    return remove_stopwords(p, stopwords=stopWords)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)

    lemmatized_tokens = [
        lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag))
        if get_wordnet_pos(tag)
        else lemmatizer.lemmatize(word)
        for word, tag in tagged_tokens
    ]
    return ' '.join(lemmatized_tokens)
stem_text
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_custom_stopwords, remove_stopwords, strip_short, lemmatize]

In [12]:
# Decode to utf-8 as needed

phishingDecoded = []
for b in phishing_text['body'].tolist():
    try:
        p = b.decode("utf-8", errors="ignore")
    except AttributeError:
        p = b
    phishingDecoded.append(p)

legitimateDecoded = []
for b in legitimate_text['body'].tolist():
    try:
        p = b.decode("utf-8", errors="ignore")
    except AttributeError:
        p = b
    legitimateDecoded.append(p)

In [13]:
# Phishing emails
phishingPreprocessed = []
for b in phishingDecoded:
    p = preprocess_string(b, filters=CUSTOM_FILTERS)
    
    phishingPreprocessed.append(p)
print(len(phishingPreprocessed))

4927


In [14]:
# legitimate emails
legitimatePreprocessed = []
for b in legitimateDecoded:
    p = preprocess_string(b, filters=CUSTOM_FILTERS)
    
    legitimatePreprocessed.append(p)
print(len(legitimatePreprocessed))

6373


In [15]:
# Lọc và loại bỏ các token list rỗng
phishingPreprocessed = [tokens for tokens in phishingPreprocessed if tokens]

# Chuyển danh sách 2 chiều thành danh sách 1 chiều với định dạng chuỗi
phishingPreprocessed = [str(tokens) for tokens in phishingPreprocessed]
phishingDataFrame = pd.DataFrame(phishingPreprocessed, columns=['body'])
phishingDataFrame['label'] = 1
phishingDataFrame = phishingDataFrame[['label', 'body']]

In [16]:
phishingDataFrame.head()

,label,body
0,1,"['hello', 'nrt', 'geeksquad®', 'support', 'tea..."
1,1,"['verify', 'mailbox', 'dear', 'mail', 'account..."
2,1,"['mailbox', 'quota', 'cause', 'mailbox', 'impa..."
3,1,"['hinton', 'dropbox', 'share', 'file', 'click'..."
4,1,"['educate', 'want', 'know', 'satellite', 'rca'..."


In [17]:
legitimatePreprocessed = [tokens for tokens in legitimatePreprocessed if tokens]
legitimatePreprocessed = [str(tokens) for tokens in legitimatePreprocessed]
legitimateDataFrame = pd.DataFrame(legitimatePreprocessed, columns=['body'])
legitimateDataFrame['label'] = 0
# Đổi thứ tự cột sao cho 'label' đứng trước 'body'
legitimateDataFrame = legitimateDataFrame[['label', 'body']]

In [18]:
legitimateDataFrame.head()

,label,body
0,0,"['matthias', 'saou', 'write', 'install', 'xmms..."
1,0,"['monday', 'september', 'student', 'life', 'be..."
2,0,"['hello', 'monday', 'september', 'write', 'thi..."
3,0,"['chris', 'garrigues', 'say', 'brent', 'welch'..."
4,0,"['slate', 'today', 'paper', 'new', 'york', 'ti..."


In [19]:
mix_dataset = pd.concat([phishingDataFrame, legitimateDataFrame], ignore_index=True)
mix_dataset = mix_dataset.sample(frac=1, random_state=1769).reset_index(drop=True)
mix_dataset.head()

,label,body
0,1,"['content', 'type', 'text', 'charset', 'iso', ..."
1,1,"['jose', 'password', 'expire', 'kindly', 'upda..."
2,0,"['new', 'software', 'feature', 'like', 'theate..."
3,1,"['message', 'trust', 'sender', 'apple', 'accou..."
4,0,"['original', 'message', 'grigsby', 'mike', 'se..."


In [20]:
raw_utils.save_to_csv(mix_dataset, csv_path, 'mix_dataset.csv')

Saving to /home/khanhnguyen/Public/workspace/python_3.10/project-phishing-email-detection/datasets/csv/mix_dataset.csv


In [21]:
train_dataset, test_dataset = util.dataset_split(mix_dataset, percent=20)

In [22]:
raw_utils.save_to_csv(train_dataset, csv_path, 'train_dataset.csv')
raw_utils.save_to_csv(test_dataset, csv_path, 'test_dataset.csv')

Saving to /home/khanhnguyen/Public/workspace/python_3.10/project-phishing-email-detection/datasets/csv/train_dataset.csv
Saving to /home/khanhnguyen/Public/workspace/python_3.10/project-phishing-email-detection/datasets/csv/test_dataset.csv
